In [ ]:
%load_ext lab_black

# Using GloVe for text classification

In this pre-filled notebook, we use GloVe embeddings to train a classifier for sentiment analysis. For every review in the IMDB dataset we:
1. Tokenize the review into tokens.
2. Get the pre-trained GloVe vector for every token in the review (if they are in the voabulary of GloVe).
3. Average the vectors over the full review.
4. Send the vector through a logistic regression.

This time, we will batch the inputs instead of updating the weights once per epoch.

Before starting, to make your experiments reproducible, make sure to [force the random seed](https://pytorch.org/docs/stable/notes/randomness.html).

In [8]:
from copy import deepcopy
from functools import partial
from typing import Callable, Dict, Generator, List, Tuple

from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import torch
from torch import nn
from torchtext import vocab
from torchtext.vocab import GloVe
from torchtext.data.utils import get_tokenizer

from tqdm.auto import tqdm

## Using GloVe (1 point)

Let's get familier with GloVe embeddings. We download a small version of GloVe trained of 6 billion words, and use vectors of size 300.

The [torchtext documentation](https://pytorch.org/text/stable/vocab.html#glove) being quite poor, you can find details on the different pre-trained vectors on the [Stanford page](https://nlp.stanford.edu/projects/glove/).

In [9]:
glove = GloVe(name="6B", dim=300)
len(glove.stoi)

.vector_cache/glove.6B.zip: 862MB [02:50, 5.05MB/s]                                                                                                                                                                
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 399999/400000 [01:12<00:00, 5503.87it/s]


400000

In [14]:
glove.stoi["cat"], glove.itos[5450], glove.vectors[glove.stoi["cat"]].shape
glove.stoi

{'the': 0,
 ',': 1,
 '.': 2,
 'of': 3,
 'to': 4,
 'and': 5,
 'in': 6,
 'a': 7,
 '"': 8,
 "'s": 9,
 'for': 10,
 '-': 11,
 'that': 12,
 'on': 13,
 'is': 14,
 'was': 15,
 'said': 16,
 'with': 17,
 'he': 18,
 'as': 19,
 'it': 20,
 'by': 21,
 'at': 22,
 '(': 23,
 ')': 24,
 'from': 25,
 'his': 26,
 "''": 27,
 '``': 28,
 'an': 29,
 'be': 30,
 'has': 31,
 'are': 32,
 'have': 33,
 'but': 34,
 'were': 35,
 'not': 36,
 'this': 37,
 'who': 38,
 'they': 39,
 'had': 40,
 'i': 41,
 'which': 42,
 'will': 43,
 'their': 44,
 ':': 45,
 'or': 46,
 'its': 47,
 'one': 48,
 'after': 49,
 'new': 50,
 'been': 51,
 'also': 52,
 'we': 53,
 'would': 54,
 'two': 55,
 'more': 56,
 "'": 57,
 'first': 58,
 'about': 59,
 'up': 60,
 'when': 61,
 'year': 62,
 'there': 63,
 'all': 64,
 '--': 65,
 'out': 66,
 'she': 67,
 'other': 68,
 'people': 69,
 "n't": 70,
 'her': 71,
 'percent': 72,
 'than': 73,
 'over': 74,
 'into': 75,
 'last': 76,
 'some': 77,
 'government': 78,
 'time': 79,
 '$': 80,
 'you': 81,
 'years': 82,
 'i

In [11]:
glove.itos[:5]

['the', ',', '.', 'of', 'to']

Notice that punctuations are part of GloVe's vocabulary.

To compare two words, we can look at their cosine similarity.

In [12]:
words = ["cat", "cats", "dog", "fridge"]
for word in words[1:]:
    similarity = torch.cosine_similarity(
        glove.vectors[glove.stoi[words[0]]].reshape(1, -1),
        glove.vectors[glove.stoi[word]].reshape(1, -1),
    ).item()  # .item() is used to turn a tensor of a single value to a float
    print(f"{words[0]} x {word} = {similarity}")

cat x cats = 0.6815836429595947
cat x dog = 0.6816746592521667
cat x fridge = 0.09630905091762543


**\[1 point\] Find the closest word to "cat" in the whole vocabulary.**

In [30]:
word = "cat"
max_sim = 0
word_max_sim = ""
# your code
for pair in glove.stoi:
    if (pair == word):
        continue;
    #print(pair)
    similarity = torch.cosine_similarity(
        glove.vectors[glove.stoi[word]].reshape(1, -1),
        glove.vectors[glove.stoi[pair]].reshape(1, -1),
    ).item()  # .item() is used to turn a tensor of a single value to a float
    if (similarity > max_sim):
        word_max_sim = pair
        max_sim = similarity
print(max_sim)
print(word_max_sim)

0.6816746592521667
dog


## Dataset and split

As we keep the test set for final evaluation, we need to split the training set into a training and validation set. We make sure the split is **stratified** by class (same proportion of class in each split).

In [31]:
dataset = load_dataset("imdb")
train_dataset = dataset["train"].train_test_split(
    stratify_by_column="label", test_size=0.2, seed=42
)
test_df = dataset["test"]
train_df = train_dataset["train"]
valid_df = train_dataset["test"]
train_df.shape, valid_df.shape, test_df.shape

Found cached dataset imdb (/home/timothee/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

((20000, 2), (5000, 2), (25000, 2))

## Prepare the inputs

### Text processing pipeline (2 points)

For a given entry, we want to
1. Tokenize the text.
2. Get the vectors for each token.
3. Average them.

For tokenization, let's use the "basic_english" tokenizer from torchtext.

In [32]:
tokenizer = get_tokenizer("basic_english", language="en")

**\[2 points\] Fill the `preprocess_text` function so it returns the mean of the GloVe vectors of all the tokens within a review.**

The two following functions can help.
* [torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html)
* [torch.mean](https://pytorch.org/docs/stable/generated/torch.mean.html)

In [61]:
def vectorize_text(
    text: str, vocabulary: vocab.Vocab, tokenizer: Callable[[str], List[str]]
) -> torch.Tensor:
    """
    Turn a string into the average of the vectors of its tokens.
    Args:
        text: the input text.
        vocabulary: a pre-trained Vocab object.
        tokenizer: a tokenizer taking a text as input and returning a list of tokens.
    Returns:
        The average tensor over the tokens of the whole text.
    """
    # Your code
    list_str = tokenizer(text)
    print(list_str)
    print(vocabulary)
    glv_vec = vocabulary.get_vecs_by_tokens(list_str)
    #print(glv_vec)
    
    return torch.mean(glv_vec, 1)

In [64]:
text ="some text:"
list_str = tokenizer(text)
glv_vec = glove.get_vecs_by_tokens(list_str)
torch.mean(glv_vec) # option of torch.mean to add to have an output that is not a real

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [63]:
shapetext_pipeline = partial(vectorize_text, vocabulary=glove, tokenizer=tokenizer)
assert text_pipeline("some text.").shape == torch.Size([300])

['some', 'text', '.']
tensor([[-2.0969e-01,  1.8073e-01, -1.6401e-01,  1.3859e-02, -1.2066e-01,
         -3.2045e-01,  3.0066e-01,  2.9993e-01,  7.9433e-02, -1.7379e+00,
         -3.3073e-01,  3.8947e-01, -3.2170e-02,  4.9593e-01,  4.2202e-01,
          1.3376e-01, -3.7729e-01,  1.9272e-01,  4.6088e-02,  1.9894e-01,
          1.3188e-01,  1.5433e-01,  3.6765e-01,  5.4241e-02, -3.2795e-01,
          1.3680e-02, -2.7777e-01,  3.4166e-01,  7.8530e-02,  4.5871e-01,
          2.3654e-01,  4.2045e-01, -6.2128e-01, -9.0873e-02, -8.3579e-01,
         -3.3465e-01,  1.7828e-01,  4.9772e-01,  1.6395e-01, -4.6531e-01,
          2.1805e-02, -4.8171e-01,  1.5414e-01, -7.1629e-02,  2.5237e-01,
          1.5213e-02, -8.8422e-03, -3.2659e-02, -5.7228e-01,  1.4588e-01,
          1.3302e-01, -3.2644e-01,  1.3202e-01,  2.0259e-01, -1.4579e-02,
          3.8435e-01,  2.6736e-02,  3.0849e-01,  4.5976e-01,  3.0015e-01,
          2.4224e-01,  1.9518e-01,  4.7310e-01,  3.5857e-02,  1.5013e-02,
         -3.9917

AssertionError: 

In [54]:
text_pipeline("some text.").shape

['some', 'text', '.']
tensor([[-2.0969e-01,  1.8073e-01, -1.6401e-01,  1.3859e-02, -1.2066e-01,
         -3.2045e-01,  3.0066e-01,  2.9993e-01,  7.9433e-02, -1.7379e+00,
         -3.3073e-01,  3.8947e-01, -3.2170e-02,  4.9593e-01,  4.2202e-01,
          1.3376e-01, -3.7729e-01,  1.9272e-01,  4.6088e-02,  1.9894e-01,
          1.3188e-01,  1.5433e-01,  3.6765e-01,  5.4241e-02, -3.2795e-01,
          1.3680e-02, -2.7777e-01,  3.4166e-01,  7.8530e-02,  4.5871e-01,
          2.3654e-01,  4.2045e-01, -6.2128e-01, -9.0873e-02, -8.3579e-01,
         -3.3465e-01,  1.7828e-01,  4.9772e-01,  1.6395e-01, -4.6531e-01,
          2.1805e-02, -4.8171e-01,  1.5414e-01, -7.1629e-02,  2.5237e-01,
          1.5213e-02, -8.8422e-03, -3.2659e-02, -5.7228e-01,  1.4588e-01,
          1.3302e-01, -3.2644e-01,  1.3202e-01,  2.0259e-01, -1.4579e-02,
          3.8435e-01,  2.6736e-02,  3.0849e-01,  4.5976e-01,  3.0015e-01,
          2.4224e-01,  1.9518e-01,  4.7310e-01,  3.5857e-02,  1.5013e-02,
         -3.9917

torch.Size([])

Now we turn our 3 sets into vectors and labels.

Our data are quite small, so we can keep everything in RAM.

In [ ]:
X_train = [text_pipeline(text) for text in tqdm(train_df["text"])]
y_train = train_df["label"]
X_valid = [text_pipeline(text) for text in tqdm(valid_df["text"])]
y_valid = valid_df["label"]
X_test = [text_pipeline(text) for text in tqdm(test_df["text"])]
y_test = test_df["label"]

### Batch processing (1 point)

Instead of doing one update per epoch, we feed the model batches of texts between each update. To do so, we use a simple data generator.

**\[1 point\] Fill the generator function.**


In [ ]:
def data_generator(
    X: List[torch.tensor], y: List[int], batch_size: int = 32
) -> Generator[Tuple[torch.Tensor, torch.Tensor], None, None]:
    """
    Yield batches from given input data and labels.
    Args:
        X: a list of tensor (input features).
        y: the corresponding labels.
        batch_size: the size of every batch [32].
    Returns:
        A tuple of tensors (features, labels).
    """
    X, y = shuffle(X, y)
    # Your code
    # yield the the returning values

In [ ]:
train_gen = lambda: data_generator(X_train, y_train, batch_size=32)
for X, y in train_gen():
    assert X.shape == torch.Size([32, 300])
    assert y.shape == torch.Size([32])
    break

## The classifier (1 point)

We create a very simple classifier corresponding a logistic regression.

**\[1 point\] Fill the classifier's code. The forward function needs to return a logit and not the output of a sigmoid.**

In [ ]:
class SimpleClassifer(nn.Module):
    """
    A simple linear classifier.
    """

    def __init__(self, embedding_size: int, nb_classes: int) -> None:
        """
        Args:
            embedding_size: the dimension of the input embeddings.
        nb_classes: the output dimension.
        """
        # your code

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: an input tensor
        Returns:
            Logits.
        """
        # your code

## Training (3 points)

We put everything above together and train the classifier.

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
train_gen = lambda: data_generator(X_train, y_train)
valid_gen = lambda: data_generator(X_valid, y_valid)
test_gen = lambda: data_generator(X_test, y_test)

**\[3 points\] Fill the following cells. Make sure you save the best model evaluated on the validation set.**
* The `deepcopy` function might help.

In [ ]:
model = SimpleClassifer(300, 1).to(device)
criterion = nn.BCEWithLogitsLoss()
# You can use another optimizer if you want.
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
nb_epochs = 50
train_losses, valid_losses = [], []

best_model = model
best_validation_loss = np.Inf

for epoch in tqdm(range(nb_epochs)):

    # training
    model.train()
    # training loop
    
    # validation
    model.eval()
    # validation loop

In [ ]:
plt.plot(train_losses, label="train loss")
plt.plot(valid_losses, label="valid loss")
plt.legend()

## Evaluation (3 point)

**\[1 point\] Compute the accuracy for the 3 splits (training, validation, test).**

In [ ]:
# Your code

**\[1 point\] For two wrongly classified samples, try guessing why the model was wrong.**

**\[1 point\] Code a `predict` function which take some text as input and returns a prediction class and score (the output of the sigmoid).**

In [ ]:
def predict(
    text: str,
    text_pipeline: Callable[[str], torch.Tensor],
    model: nn.Module,
    device: str,
) -> Tuple[int, float]:
    """
    Return the predicted class and score for a given input.
    Args:
        text: a given review.
        text_pipeline: a function taking a text as input and returning a tensor (model's input).
        model: a pre-trained model.
        device: the device on which the computation occurs.
    Returns:
        A tuple (label, score).
    """
    # Your code

In [ ]:
text = "In my long years as a movie reviewers, I have seen good and bad movies. But nothing as controversially in the middle."
predict(text, text_pipeline, model, device)

## Bonus

Modify the classifier. Instead of using a simple logistic regression, create a multilayer perceptron. Something like `input -> linear(embedding_size, 128) -> activation function -> linear(128, nb_classes) -> output`, for a two layer perceptron.

For the activation function, you can use [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#torch.nn.ReLU) or [another non-linear activation function](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity) of your choice.

Train your new classifier, look at the loss, and compare its accuracy with the logistic regression. Keep the model with the best validation loss.